# Mutual Funds Analysis
Release  V0.2.0

# README

The project can be found at
https://github.com/rishabhrkaushik/MF-Investment-Analyser

## Index

 - [Purpose of the Project](#purpose-of-the-project)
 - [Disclaimer](#disclaimer)
 - [Features](#features)
 - [Getting Started](#getting-started)
 - [Get In Touch](#get-in-touch)
 - [Author](#author)

## Purpose of the Project
The purpose of the project is to analyze one's Mutual Funds Portfolio. The project involves creating various graphs and summary table to give insights on one's investments so as to take informed decisions on investment and disinvestment.  The graphs and summary table are preliminary and investor should take consider various other factors before taking any decisions.

*Note: This is not completely automated script. The investment details needs to be stored manually in a JSON file.*

## Disclaimer
`I am in no way an expert on investments. Mutual Fund investments are subject to market risks. This project helps me track my portfolio. Do not use this as a starting point for your investments. Read various terms related to investment and talk to some knowledgeable person before starting investment.
This project can be used to view good visualizations and study the trends. `

## Features
The project at present contains following features.

 1. [Point of investment graph](#point-of-investment-graph)
 2. [Current Value vs Invested Value Graph](#current-value-vs-invested-value-graph)
 3. [Summary Table](#summary-table)

### Point of Investment Graph
![Point of Investment Graph Screenshot](https://raw.githubusercontent.com/rishabhrkaushik/MF-Investment-Analyser/master/Screenshots/Point of Investment Graph.png)
### Current Value vs Invested Value Graph
![Current Value vs Invested Amount Screenshot](https://raw.githubusercontent.com/rishabhrkaushik/MF-Investment-Analyser/master/Screenshots/Current Value vs Invested Amount.png)
### Summary Table
![Summary Screenshot](https://raw.githubusercontent.com/rishabhrkaushik/MF-Investment-Analyser/master/Screenshots/Summary Table.png)
## Getting Started
The project can be run either in your local environment or on Google Colab. The later one gives you ability to access your files from anywhere and require minimal setup. Refer `Getting Started.md` file in docs for the same.

## Get in Touch

Feel free to contact me at `rishabhrkaushik@gmail.com` with subject starting with `Github MF Analyzer Query | `

## Author
Rishabh Kaushik
https://rishabhrkaushik.github.io

# Code

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Import Libraries

### Import Data Manipulation Libraries

In [ ]:
import pandas as pd
import numpy as np

### Import data visualization libraries

In [ ]:
import matplotlib
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import plot

### Access Google Drive
Only required if running environment is Colab and JSON is stored in Drive.

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')

### Import std libs

In [ ]:
import json
import pprint
pp = pprint.PrettyPrinter(indent = 2)

### Import datetime and get today as variable

In [ ]:
from datetime import datetime
from datetime import timedelta
now = datetime.now()
today = now.strftime("%Y-%m-%d")

### Import Quandl
Financial data API

In [ ]:
# only for google colab environment
# !pip3 install quandl

In [ ]:
import quandl

## Load JSON

In [ ]:
# FilePath = "gdrive/My Drive/Colab Notebooks/Mutual Funds Analysis/"
FilePath = "./../Data/"
FileName = "MF Portfolio.json"
with open(FilePath + FileName) as data:
    investmentDetails = json.load(data)
    
print(investmentDetails['Account Holder Name'])

## Configure Quandl

In [ ]:
quandl.ApiConfig.api_key = investmentDetails['Quandl API Key']

## Function to get current invested amount, units and current value

In [ ]:
def addCurrentHoldings(row, transaction):
    if(row.name.to_pydatetime().date() in transaction["Date"].values):
#         print(row.name.to_pydatetime().date())
        row["Holding Units"] = transaction.loc[transaction["Date"] == row.name.to_pydatetime().date()]["Units"]
        row["Current Invested Amount"] = transaction.loc[transaction["Date"] == row.name.to_pydatetime().date()]["Amount"] 
#         print(row)
    return row

 ## Create Dataframe out of investments JSON

In [ ]:
start_date="2000-01-01"
fundsDetails = []
for fund in investmentDetails["Funds"]:
    navs = quandl.get(fund["ID"], start_date = start_date, end_date = today)
    navs.drop(["Repurchase Price", "Sale Price"], axis = 1, inplace=True)
    
    transactionDates = []
    transactionUnits = []
    transactionAmounts = []
    
    purchasingDates = []
    purchasedUnits = []
    purchasedAmounts = []
    
    sellingDates = []
    soldUnits = []
    soldAmounts = []
    
    for transaction in fund["Transactions"]:
        transactionDates.append(datetime.strptime(transaction["Date"], "%Y-%m-%d").date())
        transactionUnits.append(transaction["Units"])
        transactionAmounts.append(transaction["Amount"])
        if(transaction["Units"] > 0):
            purchasingDates.append(datetime.strptime(transaction["Date"], "%Y-%m-%d").date())
            purchasedUnits.append(transaction["Units"])
            purchasedAmounts.append(transaction["Amount"])
        else:
            sellingDates.append(datetime.strptime(transaction["Date"], "%Y-%m-%d").date())
            soldUnits.append(-transaction["Units"])
            soldAmounts.append(-transaction["Amount"])
            
    transactions = pd.DataFrame({
        "Date": transactionDates, 
        "Units": transactionUnits, 
        "Amount": transactionAmounts
    })
    
    purchases = pd.DataFrame({
        "Date": purchasingDates, 
        "Units": purchasedUnits, 
        "Amount": purchasedAmounts
    })
        
    sales = pd.DataFrame({
        "Date": sellingDates, 
        "Units": soldUnits, 
        "Amount": soldAmounts
    })
    
    fundsDetails.append({
        "ID": fund["ID"],
        "Name": fund["Fund Name"], 
        "Tags": fund["Tags"],
        "Lock In": fund["Lock In"]["Period"],
        "Short Term Tax Implication": fund["Tax Implication"]["Short Term"],
        "Long Term Tax Implication": fund["Tax Implication"]["Long Term"],
        "Exit Load - Period": fund["Exit Load"]["Period"],
        "Exit Load - Load": fund["Exit Load"]["Load"],
        "Navs": navs, 
        "Transactions": transactions,
        "Purchases" : purchases,
        "Sales": sales
    })

In [ ]:
for fund in fundsDetails:
#     print(fund["name"])
    noOfElements = fund["Navs"].count().max()
    fund["Navs"]["Holding Units"] = [0] * noOfElements
    fund["Navs"]["Current Invested Amount"] = [0] * noOfElements
    fund["Navs"]["Current Value"] = [0] * noOfElements
    fund["Navs"] = fund["Navs"].apply(addCurrentHoldings, transaction = fund["Transactions"], axis='columns')
    fund["Navs"]["Holding Units"] =fund["Navs"]["Holding Units"].cumsum()
    fund["Navs"]["Current Invested Amount"] = fund["Navs"]["Current Invested Amount"].cumsum()
    fund["Navs"]["Current Value"] = fund["Navs"]["Holding Units"] * fund["Navs"]["Net Asset Value"]
    fund.update({"Navs": fund["Navs"]})

In [ ]:
rows = []
for fund in fundsDetails:
    fundDetail = {}
    fundDetail["ID"] = fund["ID"]
    fundDetail["Fund Name"] = fund["Name"]
    fundDetail["Current Investment"] = fund["Transactions"]["Amount"].sum()
    fundDetail["Current Units"] = fund["Transactions"]["Units"].sum()
    fundDetail["Current NAV"] = fund["Navs"]["Net Asset Value"].iat[-1]
#     print(fundDetail)
    fundDetail["Current Value"] = fundDetail["Current NAV"] * fundDetail["Current Units"]
    fundDetail["Profit/ Loss"] = (
                                    fundDetail["Current Value"]
                                    + fund["Sales"]["Amount"].sum()
                                    - fund["Purchases"]["Amount"].sum()
    )
    fundDetail["Percentage Growth"] = 100 * fundDetail["Profit/ Loss"]/fund["Purchases"]["Amount"].sum()
    fundDetail["Average Purchase NAV"] = fund["Purchases"]["Amount"].sum()/fund["Purchases"]["Units"].sum()
    fundDetail["First Investment"] = fund["Transactions"]["Date"].sort_values().iloc[0]
    fundDetail["Last Investment"] = fund["Transactions"]["Date"].sort_values().iloc[-1]
    fundDetail["Tags"] = fund["Tags"]
    fundDetail["Lock In"] = fund["Lock In"]
    fundDetail["Short Term Tax Implication"] = fund["Short Term Tax Implication"]
    fundDetail["Long Term Tax Implication"] = fund["Long Term Tax Implication"]
    fundDetail["Exit Load - Period"] = fund["Exit Load - Period"]
    fundDetail["Exit Load - Load"] = fund["Exit Load - Load"]
    
    rows.append(fundDetail)
investmentSummary = pd.DataFrame(rows)[[
                        "ID",
                        "Fund Name", 
                        "Current Value", 
                        "Current Investment",  
#                         "Current Units", 
#                         "Current NAV", 
                        "Profit/ Loss", 
                        "Percentage Growth",
#                         "Average Purchase NAV",
                        "First Investment",
                        "Last Investment",
                        "Tags",
                        "Lock In",
                        "Short Term Tax Implication",
                        "Long Term Tax Implication",
                        "Exit Load - Period",
                        "Exit Load - Load"
                    ]]

## Dataframe structure of investmentSummary
```json
[[
    "Fund Name", 
    "Current Units", 
    "Current NAV", 
    "Current Investment",  
    "Current Value", 
    "Profit/ Loss", 
    "Percetage Profit",
    "Average Purchase NAV",
    "First Investment",
    "Last Investment",
    "Tags",
    "Lock In",
    "Short Term Tax Implication",
    "Long Term Tax Implication",
    "Exit Load - Period",
    "Exit Load - Load"
]]
```

## Create Point of Investment Graphs

In [ ]:
pointsOfTransactions = []
for fund in fundsDetails:
    fig = make_subplots(specs = [[{"secondary_y": True}]])
    fig.add_trace(
        go.Scatter(x = fund["Navs"].index, 
                   y = fund["Navs"]['Net Asset Value'], 
                   name = "NAV", 
                   line_color = 'deepskyblue'
                  )
    )
    fig.add_trace(
        go.Bar(
            x = fund["Purchases"]["Date"],
            y = fund["Purchases"]["Units"],
            width = 1000*3600*24, #width is counted in milliseconds
            name = "Purchases"
        ), 
        secondary_y = True
    )
    
    fig.add_trace(
        go.Bar(
            x = fund["Sales"]["Date"],
            y = fund["Sales"]["Units"],
            width = 1000*3600*24, #width is counted in milliseconds
            name = "Sales"
        ), 
        secondary_y = True
    )

    fig.update_xaxes(title_text="Date")

    fig.update_yaxes(title_text="Amount (Rs.)", secondary_y=False)
    fig.update_yaxes(title_text="Units", secondary_y=True)

    fig.update_layout(
        title_text = fund["Name"]
    )
    
    pointsOfTransactions.append(fig)

## Create Current Value vs Invested Amount Graphs

In [ ]:
investmentVsValue = []
for fund in fundsDetails:
    fig = go.Figure()
    fig.add_trace(
        go.Scatter(x = fund["Navs"].index, 
                   y = fund["Navs"]['Current Invested Amount'], 
                   name = "Invested", 
                   line_color = 'deepskyblue'
                  )
    )
    
    fig.add_trace(
        go.Scatter(x = fund["Navs"].index, 
                   y = fund["Navs"]['Current Value'], 
                   name = "Current Value", 
                   line_color = 'red'
                  )
    )
    
    fig.update_xaxes(title_text="Date")
    fig.update_yaxes(title_text="Amount (Rs.)")

    fig.update_layout(
        title_text = fund["Name"]
    )
        
    investmentVsValue.append(fig)

# Helper Function
## Invested Before
pass ID of fund as string and date in "YYYY-MM-DD" format as string

Returns number of unit holding before a particular date

In [ ]:
def investedBefore(fundId, date):
    fundIndex = next((index for (index, d) in enumerate(fundsDetails) if d["ID"] == fundId), None)
    holdings = 0
#     print(fundsDetails[fundIndex]["Transactions"])
    for ind in fundsDetails[fundIndex]["Transactions"].index:
#         print(ind)
        transaction = fundsDetails[fundIndex]["Transactions"].iloc[ind]
        if(transaction["Date"] < datetime.strptime(date, "%Y-%m-%d").date()):
            holdings = holdings + transaction["Units"]
    return holdings

## Get Value
pass ID of fund as string and no of units in int

Return value of units at current time

In [ ]:
def getValue(fundId, units):
    fundIndex = next((index for (index, d) in enumerate(fundsDetails) if d["ID"] == fundId), None)
    value = units * fundsDetails[fundIndex]["Navs"]["Net Asset Value"].iat[-1]
    return value

## Viewing Ground

In [ ]:
investedBefore("AMFI/118530", "2019-07-08")
getValue("AMFI/118530", investedBefore("AMFI/118530", "2019-07-08"))

### Summary Table

In [ ]:
investmentSummary.round(decimals=2)
print("Profit/ Loss: ", investmentSummary["Profit/ Loss"].sum())
print("Current Investment: ", investmentSummary["Current Investment"].sum())

### Graphs

In [ ]:
# xaxis_range = [datetime(2010, 1, 1), now]
xaxis_range = [now - timedelta(days=15), now]

### View Points of Transaction Graph

In [ ]:
for figure in pointsOfTransactions:
    figure.update_layout(
        xaxis_range = xaxis_range
    )
    figure.show()

### View Current Value vs Invested Amount Graphs

In [ ]:
for figure in investmentVsValue:
    figure.update_layout(
        xaxis_range = xaxis_range
    )
    figure.show()